# Start using the spotify API

In [2]:
import config
import spotipy
import json
import requests
from spotipy.oauth2 import SpotifyClientCredentials
import pprint
import pandas as pd
import base64
import csv
import pickle

In [3]:
# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(
    client_id=config.client_id,
    client_secret=config.client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
results = sp.search(q="Lose yourself",limit=3,market="GB")

In [6]:
track_id=results["tracks"]["items"][0]["id"]
track_id

'1v7L65Lzy0j0vdpRjJewt1'

In [ ]:
# Get "Today's Top Hits" playlist ID
playlist_id = "37i9dQZF1DXcBWIGoYBM5M"

# Get top tracks and show only the ids
playlist_items = sp.playlist_items(playlist_id, fields="items.track.id", limit=10)

track_id = []
track_features = []
# Print top tracks
for i, item in enumerate(playlist_items["items"]):
    # get track_id
    track_id = item["track"]["id"]
    # get features and store them in a list
    track_features.append(sp.audio_features(track_id)[0])

# create a dataframe from the features list
df = pd.DataFrame(track_features)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
# create a list containing all markets
markets = list(sp.available_markets()["markets"])

In [ ]:
# get list of category IDs for all the markets
category_id_lst = []
for market in markets:
    # get list of category IDs for a certain country
    categories_per_country = sp.categories(country=market, limit=50)["categories"]["items"]
    # get to the ID's of the categories and store them in a list
    for category in categories_per_country:
        if not category["id"] in category_id_lst:
            category_id_lst.append(category["id"])
        else:
            print("already in list")

In [ ]:
# create a list of playlist IDs from all the categories of all the markets
playlist_id_lst = []
for category_id in category_id_lst:
    try:
        playlists_per_genre = sp.category_playlists(category_id=category_id, limit=50)["playlists"]["items"]
        try:
            for playlist in playlists_per_genre:
                if not playlist["id"] in playlist_id_lst:
                    playlist_id_lst.append(playlist["id"])
                else:
                    pass
        except:
            nothing = True
    except spotipy.SpotifyException as e:
        pass

In [ ]:
df_playlists = pd.DataFrame(playlist_id_lst)
df_playlists.to_csv("list_of_playlists.csv", index_label=False)

In [3]:
user = "spotify"
# define function to get tracks from a playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(user=username,playlist_id=playlist_id, fields="items.track.id, next", limit=10)
    print(results)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
df_tracks = pd.DataFrame(columns=["ID"])
for track in tracks:
    try:
        df_tracks.loc[len(df_tracks)] = track["track"]["id"]
    except:
        pass

df_tracks.to_csv("df_tracks.csv", index_label=False)

In [ ]:
tracks = []
for id in playlist_id_lst:
    tracks.extend(get_playlist_tracks(user, id))

In [5]:
# load tracks from csv into list
df_tracks = pd.read_csv("df_tracks.csv")
tracks = df_tracks.values.tolist()

In [6]:
len(tracks)

102483

In [55]:
def get_features(track_id):
    track_features = []
    track_features.extend(sp.audio_features(track_id))
    return track_features

In [8]:
def write_list_to_pickle_file(my_list, filename):
    with open(filename, 'wb') as file:
        pickle.dump(my_list, file)

In [9]:
def write_list_to_csv(my_list, filename):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        for item in my_list:
            writer.writerow([item])

In [ ]:
for track in tracks[0:20000]:
    try:
        track_features_lst = []
        track_features_lst.extend(get_features(track))
        write_list_to_csv(track_features_lst, "track_features_lst.csv")
    except:
        pass

with open("track_features_lst_2.csv","rb") as f:
    features = pickle.load(f)

In [59]:
df = pd.read_csv("track_features_lst_2.csv")

df_features = pd.DataFrame(columns=["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"])
df_features

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms


In [34]:
# example DataFrame with dictionaries as rows
df = pd.read_csv("track_features_lst_2.csv")


In [ ]:
track_features = []
for playlist_id in playlist_id_lst[0:20]:
    try:
        playlist_items = sp.user_playlist_tracks(user=user, playlist_id=playlist_id, fields="items.track.id, next", limit=10)
    except:
        pass

    try:
        tracks = playlist_items['items']
        while playlist_items['next']:
            playlist_items = sp.next(playlist_items)
            tracks.extend(playlist_items['items'])
            track_features.append(sp.audio_features(tracks)[0])
    except:
        pass

# create a dataframe from the features list
df = pd.DataFrame(track_features)
#df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
track_features

In [ ]:
# Get top tracks from all the playlists
for playlist_id in playlist_id_lst:
    playlist_items = sp.playlist_items(playlist_id, fields="items.track.id", limit=1)

    # now get the ID's
    track_id = []
    track_features = []
    # Print top tracks
    for i, item in enumerate(playlist_items["items"]):
        try:
            # get track_id
            track_id = item["track"]["id"]
            # get features and store them in a list
            track_features.append(sp.audio_features(track_id)[0])
        except TypeError:
            print("type error")

# create a dataframe from the features listS
df = pd.DataFrame(track_features)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
df.head()

# code snippets

In [ ]:
# Set up authorization headers
client_id = config.client_id
client_secret = config.client_secret
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode("ascii")).decode("ascii")
headers = {"Authorization": f"Basic {auth_header}"}

# Get access token
url = "https://accounts.spotify.com/api/token"
data = {"grant_type": "client_credentials"}
response = requests.post(url, headers=headers, data=data)
access_token = response.json()["access_token"]

# Make request to Top Tracks endpoint
artist_id = "0OdUWJ0sBjDrqHygGUXeCF"  # example artist ID
url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
headers = {"Authorization": f"Bearer {access_token}"}
params = {"market": "US"}

response = requests.get(url, headers=headers, params=params)